# Download and store images

Create dataset by assembling and resizing images

## Config

In [64]:
import pandas as pd
import numpy as np
from simple_image_download import simple_image_download as simp
import shutil
from glob import glob
from pathlib import Path
from PIL import Image, UnidentifiedImageError
from typing import Tuple, Dict
from tqdm import tqdm
import logging
import matplotlib.pyplot as plt
import seaborn as sns

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.propagate = False

INFO:matplotlib.font_manager:Generating new fontManager, this may take some time...


In [44]:
n_positive_examples = 2500
n_negative_examples = 2500

positive_terms = ["poison ivy plant"]

default_download_path = Path("simple_images")
store_path = Path("../datasets/pipeline_v1/downloaded_images")
metadata_loc = Path("../datasets/pipeline_v1/raw_downloads/image_metadata.csv")

similar_plants = [
    "Box elder",
    "Virgin's bower",
    "Virginia creeper",
    "Western poison oak",
    "Poison sumac",
    "Kudzu",
    "Blackberries",
    "Strawberries",
    "Fragrant sumac",
    "Hoptree",
] # Source: "https://en.wikipedia.org/wiki/Toxicodendron_radicans"
negative_terms = [f"{plant} plant" for plant in similar_plants]

In [11]:
simp.simple_image_download().download(positive_terms[0], 1)

In [46]:
def resize_by_min_dim(size: Tuple[int, int], min_dim:int =100) -> Tuple[int, int]:
    """
    Resize an image to keep the same proportions and make the smaller
    of the two dimensions equal to min_dim
    size = (width, height) in pixels
    returns (new_width, new_height)
    """
    width, height = size
    if (width <= min_dim) or (height <= min_dim):  # No need to resize images which are already small enough
        return size
    new_height = new_width = None
    if width > height:
        new_height = min_dim
        new_width = int((width/height) * new_height)
    else:
        new_width = min_dim
        new_height = int((height/width) * new_width)
    return (new_width, new_height)
    

def download_images(search_term: str, count: int = 5):
    """
    Downloads images based on search terms to simple_images folder
    """
    # Clear download space so we only keep downloaded images
    src_folder = Path("simple_images")  # default property of simple_image_download package
    if src_folder.exists():
        shutil.rmtree(str(src_folder))
    downloader = simp.simple_image_download()
    logging.info(f"Starting download for: {search_term} x {count}")
    downloader.download(search_term, count)
    logging.info(f"Downloaded images for: {search_term} x {count}")
        
    
    

In [58]:
def gather_and_store_image_script(search_term_counts: Dict[str, int],
                                  dest_dir: str,
                                  img_min_dimen: int = 500,
                                  clean_dest_dir: bool = False) -> pd.DataFrame:
    """
    Downloads google image based on a dict of search terms: count of images to download
    Downsizes image if they are too large by putting smaller dimension to min_dimen and keeping aspect ratio
    If clean_dest_dir, remove all data from destination first
    Returns a dataframe of the results
    """
    dest_path = Path(dest_dir)
    if dest_path.exists() and (clean_dest_dir == True):
      shutil.rmtree(dest_path)
      dest_path.mkdir(parents=True, exist_ok=True)
    if not dest_path.exists():
      dest_path.mkdir(parents=True, exist_ok=True)
    
    download_dir = Path("simple_images")
    image_metadata = []
    img_data = {}
    
    
    for search_term, count in search_term_counts.items():
      download_images(search_term, count)
      for img_loc in glob(str(Path(download_dir, search_term, '*'))):
        img_path = Path(img_loc)
        file_name = img_path.name.replace(" ", "_")
        dest_loc = Path(dest_path, file_name)
        
        img_data = {
          "download_loc": str(img_path),
          "final_loc": str(dest_loc),
          "full_path": str(dest_loc.absolute().resolve()),
          "search_term": search_term,
          "source": "Google Images",
          "file_name": file_name
        }
        
        try:
          img = Image.open(img_path).convert('RGB')
          img_data['read'] = True
        except UnidentifiedImageError:
          img_data['read'] = False
        
        if img_data['read']:
          img_data['orig_width'], img_data['orig_height'] = img.size
          img_data['width'], img_data['height'] = resize_by_min_dim((img_data['orig_width'],
                                                                     img_data['orig_height']), min_dim=500)
          resized_img = img.resize((img_data['width'], img_data['height']), Image.ANTIALIAS)
          resized_img.save(img_data['final_loc'], 'JPEG', quality=90)
        
        image_metadata.append(img_data)
    if download_dir.exists():
      shutil.rmtree(str(download_dir))

    return(pd.DataFrame(image_metadata).set_index("file_name"))
    

In [55]:
# Positive images
n_positive_each = int(n_positive_examples / len(positive_terms))
df_pos = gather_and_store_image_script({pos_term: n_positive_each
                                        for pos_term in positive_terms},
                                       str(Path(store_path, 'positive')))
df_pos['label'] = 1
df_pos['label_str'] = 'positive'

INFO:root:Downloaded all images
INFO:root:Starting download for: poison ivy plant x 2500


HTTPSConnectionPool(host='naturework.com', port=443): Max retries exceeded with url: /wp-content/uploads/2013/06/Poison_Ivy-Early-May_coloring.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f3a41d86be0>: Failed to establish a new connection: [Errno 111] Connection refused',))
HTTPSConnectionPool(host='naturework.com', port=443): Max retries exceeded with url: /wp-content/uploads/2013/06/Poison_Ivy-Early-May_coloring.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f3a41ce59b0>: Failed to establish a new connection: [Errno 111] Connection refused',))
HTTPSConnectionPool(host='naturework.com', port=443): Max retries exceeded with url: /wp-content/uploads/2013/06/Poison_Ivy-Early-May_coloring.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f3a41ce7f98>: Failed to establish a new connection: [Errno 111] Connection refused',))
HTTPSConnectionPool(host='naturework.com', port=443): Max 

INFO:root:Downloaded images for: poison ivy plant x 2500
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:976: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [59]:
# Negative images
n_negative_each = int(n_negative_examples / len(negative_terms))
df_neg = gather_and_store_image_script({neg_term: n_negative_each
                                        for neg_term in negative_terms},
                                       str(Path(store_path, 'negative')),
                                       clean_dest_dir=False)
df_neg['label'] = 0
df_neg['label_str'] = 'negative'

INFO:root:Starting download for: Box elder plant x 250
INFO:root:Downloaded images for: Box elder plant x 250
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:976: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
INFO:root:Starting download for: Virgin's bower plant x 250
INFO:root:Downloaded images for: Virgin's bower plant x 250
INFO:root:Starting download for: Virginia creeper plant x 250


('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))


INFO:root:Downloaded images for: Virginia creeper plant x 250
INFO:root:Starting download for: Western poison oak plant x 250


HTTPSConnectionPool(host='cleverhomestead.com', port=443): Max retries exceeded with url: /wp-content/uploads/2013/12/poison-ivy-eastern-western-toxicodendron-radicans-rydbergii-featured-1280x640.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f3a4cfe1a58>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))


INFO:root:Downloaded images for: Western poison oak plant x 250
INFO:root:Starting download for: Poison sumac plant x 250
INFO:root:Downloaded images for: Poison sumac plant x 250
INFO:root:Starting download for: Kudzu plant x 250


...

HTTPSConnectionPool(host='64.209.142.95', port=443): Max retries exceeded with url: /images/products/rhus_lacette.jpg (Caused by SSLError(CertificateError("hostname '64.209.142.95' doesn't match either of 'www.estabrooksonline.com', 'estabrooksonline.com'",),))


HTTPSConnectionPool(host='64.209.142.95', port=443): Max retries exceeded with url: /images/products/rhus_lacette.jpg (Caused by SSLError(CertificateError("hostname '64.209.142.95' doesn't match either of 'www.estabrooksonline.com', 'estabrooksonline.com'",),))


INFO:root:Downloaded images for: Fragrant sumac plant x 250
INFO:root:Starting download for: Hoptree plant x 250


HTTPSConnectionPool(host='centraldata.gardenconnect.com', port=443): Max retries exceeded with url: /files/images/plantguide/plant_3011dda0aa_n.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f3a4cf38cf8>: Failed to establish a new connection: [Errno 111] Connection refused',))
HTTPSConnectionPool(host='centraldata.gardenconnect.com', port=443): Max retries exceeded with url: /files/images/plantguide/plant_3011dda0aa_n.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f3a4cee1748>: Failed to establish a new connection: [Errno 111] Connection refused',))
HTTPSConnectionPool(host='centraldata.gardenconnect.com', port=443): Max retries exceeded with url: /files/images/plantguide/plant_3011dda0aa_n.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f3a4cf385c0>: Failed to establish a new connection: [Errno 111] Connection refused',))


INFO:root:Downloaded images for: Hoptree plant x 250


## Supplement with ImageNet and PlantNet data 

In [60]:
# TODO

## Analyze and store metadata

In [63]:
df_ims = pd.concat([df_pos, df_neg])
df_ims.head()

,download_loc,final_loc,full_path,search_term,source,read,orig_width,orig_height,width,height,label,label_str
file_name,,,,,,,,,,,,
poison_ivy_plant_1.jpg,simple_images/poison ivy plant/poison ivy plan...,../datasets/pipeline_v1/downloaded_images/posi...,/home/code/datasets/pipeline_v1/downloaded_ima...,poison ivy plant,Google Images,True,120.0,44.0,120.0,44.0,1,positive
poison_ivy_plant_10.jpg,simple_images/poison ivy plant/poison ivy plan...,../datasets/pipeline_v1/downloaded_images/posi...,/home/code/datasets/pipeline_v1/downloaded_ima...,poison ivy plant,Google Images,True,800.0,450.0,800.0,450.0,1,positive
poison_ivy_plant_100.jpg,simple_images/poison ivy plant/poison ivy plan...,../datasets/pipeline_v1/downloaded_images/posi...,/home/code/datasets/pipeline_v1/downloaded_ima...,poison ivy plant,Google Images,True,4250.0,2833.0,750.0,500.0,1,positive
poison_ivy_plant_1000.jpg,simple_images/poison ivy plant/poison ivy plan...,../datasets/pipeline_v1/downloaded_images/posi...,/home/code/datasets/pipeline_v1/downloaded_ima...,poison ivy plant,Google Images,True,722.0,406.0,722.0,406.0,1,positive
poison_ivy_plant_1001.jpg,simple_images/poison ivy plant/poison ivy plan...,../datasets/pipeline_v1/downloaded_images/posi...,/home/code/datasets/pipeline_v1/downloaded_ima...,poison ivy plant,Google Images,True,1280.0,720.0,888.0,500.0,1,positive
